In [1]:
import sgp4.api
from sgp4.api import Satrec, jday
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from astropy.coordinates import GCRS, ITRS
from astropy.time import Time
import astropy.units as u

In [29]:
# get tle data and convert it to sgp4 usable format
def initialisation(tlefile:str):
    with open(tlefile,"r") as file:
        lines = file.readlines()

    if len(lines)%3 !=0:
        return print("Error in TLE file format")

    tle_dict_show={}
    tle_dict_use={}
    for i in range(0,len(lines),3):
        name = lines[i].strip()
        line1 = lines[i+1].strip()
        line2 = lines[i+2].strip()
        tle_dict_show[name] = [line1,line2]
        sat = Satrec.twoline2rv(line1,line2)
        tle_dict_use[name] = sat
    
    return tle_dict_show,tle_dict_use

# using the converted format, calculate the orbit for a given time
def calculation_all_satellites(tledata, start_time:dt.datetime, end_time:dt.datetime,steps:np.float32):
    timestamps = np.arange(start_time, end_time, dt.timedelta(seconds=steps)).tolist()
    julian=[]
    for i in timestamps:
        jd,fr = jday(year = i.year,
                     mon = i.month,
                     day = i.day,
                     hr = i.hour,
                     minute = i.minute,
                     sec = i.second)
        julian.append((jd,fr))
    
    full_data_dict = {}

    for k, v in tledata.items():
        temp = []
        for j in julian:
            error, position, velocity = v.sgp4(j[0],j[1])
            arr = np.empty(6, dtype=np.float32)
            if error ==0:
                arr[0:3] = position
                arr[3:6] = velocity

                time = Time((j[0] + j[1])/86400.0, format='jd',scale='utc')
                gcrs = GCRS(obstime=time,
                            representation_type='cartesian',
                            x = arr[0]*u.km,
                            y = arr[1]*u.km,
                            z = arr[2]*u.km,
                            v_x = arr[3]*u.km/u.s,
                            v_y = arr[4]*u.km/u.s,
                            v_z = arr[5]*u.km/u.s)
                
                itrs = gcrs.transform_to(ITRS(obstime=time))
                
                arr[0] = itrs.x.to(u.km).value
                arr[1] = itrs.y.to(u.km).value
                arr[2] = itrs.z.to(u.km).value
                arr[3] = itrs.v_x.to(u.km/u.s).value
                arr[4] = itrs.v_y.to(u.km/u.s).value
                arr[5] = itrs.v_z.to(u.km/u.s).value
            else: print(error)
            temp.append(arr)
        full_data_dict[k] = temp
         
    return full_data_dict

def calculation_one_satellite(tledata, start_time:dt.datetime, end_time:dt.datetime,steps:np.float32):
    timestamps = np.arange(start_time, end_time, dt.timedelta(seconds=steps)).tolist()
    julian=[]
    for i in timestamps:
        jd,fr = jday(year = i.year,
                     mon = i.month,
                     day = i.day,
                     hr = i.hour,
                     minute = i.minute,
                     sec = i.second)
        julian.append((jd,fr))
    
    pos=[]
    vel=[]
    for j in julian:
        error,position,velocity = tledata.sgp4(j[0],j[1])
        arr1 = np.empty(3 , dtype=np.float32)
        arr2 = np.empty(3 , dtype=np.float32)
        if error == 0:
            arr1[0:3] = position
            arr2[0:3] = velocity

            time = Time((j[0] + j[1])/86400.0, format='jd',scale='utc')
            gcrs = GCRS(obstime=time,
                        representation_type='cartesian',
                        x = arr1[0]*u.km,
                        y = arr1[1]*u.km,
                        z = arr1[2]*u.km,
                        v_x = arr2[0]*u.km/u.s,
                        v_y = arr2[1]*u.km/u.s,
                        v_z = arr2[2]*u.km/u.s)
            
            itrs = gcrs.transform_to(ITRS(obstime=time))
            
            arr1[0] = itrs.x.to(u.km).value
            arr1[1] = itrs.y.to(u.km).value
            arr1[2] = itrs.z.to(u.km).value
            arr2[0] = itrs.v_x.to(u.km/u.s).value
            arr2[1] = itrs.v_y.to(u.km/u.s).value
            arr2[2] = itrs.v_z.to(u.km/u.s).value
        else: print(error)
        pos.append(arr1)
        vel.append(arr2)
    return (np.array(pos),np.array(vel))

def convert_geodetic_to_ECEF(latitude,longitude,height):
    a = 6378 #km Equatorial Radius
    b = 6357 #km Polar Radius
    e_square = 1-(b**2/a**2)
    f = 1 - (b/a)
    N_phi = a/np.sqrt(1-(e_square)*(np.sin(latitude))**2)
    X = (N_phi+height)*np.cos(latitude)*np.cos(longitude)
    Y = (N_phi+height)*np.cos(latitude)*np.sin(longitude)
    Z = (((b/a)**2)*N_phi+height)*np.sin(latitude)
    return np.array([X,Y,Z])

def visible_satellite_and_doppler(lat,long,h,satellite_pos,satellite_vel,elevation):

    c = 299792458/1000

    ground_station = convert_geodetic_to_ECEF(lat,long,h)

    vec_station_satellite = satellite_pos - ground_station
    ground_unit = ground_station/(np.sqrt(np.sum(ground_station[:]**2)))
    vec_satellite_station_mag = np.sqrt(np.sum(vec_station_satellite**2,axis=1))
    satellite_unit = vec_station_satellite/vec_satellite_station_mag
    elev = np.arcsin(np.sum(satellite_unit*ground_unit,axis=1))
    visible_pos = satellite_pos[elev>=elevation]
    visible_vel = satellite_vel[elev>=elevation]
    visible_elev = elev[elev>=elevation]
    visible_R = satellite_unit[elev>=elevation]
    radial_vel = visible_vel*visible_R
    doppler = (-1/c)*np.sum(radial_vel,axis=1)
    return (visible_pos,visible_elev,doppler)


In [ ]:
data = initialisation('stations.txt')
start = dt.datetime(2025,2,10,0,0,0)
end = dt.datetime(2025,2,10,0,0,0)
pos,vel = calculation_one_satellite(data[1]['ISS (ZARYA)'],start,end,1)

c:\Users\kusha\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:133: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
c:\Users\kusha\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:133: ErfaWarning: ERFA function "utcut1" yielded 1 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
c:\Users\kusha\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:133: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)


KeyboardInterrupt: 

In [13]:
values

array([[ 3.4166777e+03,  3.6994604e+03,  4.5603755e+03, -5.8890867e+00,
        -1.9104913e-01,  4.5604424e+00],
       [ 3.4105166e+03,  3.6995164e+03,  4.5649331e+03, -5.8934259e+00,
        -1.9530720e-01,  4.5546327e+00],
       [ 3.4043513e+03,  3.6995671e+03,  4.5694849e+03, -5.8977580e+00,
        -1.9956459e-01,  4.5488172e+00],
       ...,
       [-7.3400677e+02,  2.9244502e+03,  6.0850806e+03, -7.0030174e+00,
        -2.5010610e+00,  3.5287932e-01],
       [-7.4122272e+02,  2.9218940e+03,  6.0854297e+03, -7.0022612e+00,
        -2.5042446e+00,  3.4513089e-01],
       [-7.4843744e+02,  2.9193335e+03,  6.0857705e+03, -7.0014958e+00,
        -2.5074251e+00,  3.3738181e-01]], dtype=float32)